# `Spotify songs` dataset normalization and SQL query

In [63]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from eralchemy import render_er
from itertools import combinations, permutations
import os
import csv
import sqlite3
pd.set_option('display.max_columns', None) # to show all columns in each df
pd.set_option('display.max_rows', 100) # to show 100 rows in each df
pd.set_option('max_colwidth', None) # to show all char for each value

## Get data

In [2]:
url = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv'
ss = pd.read_csv(url)

In [3]:
ss

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxury Remix,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury Remix],2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.748,0.916,6,-2.634,1,0.0583,0.102000,0.000000,0.0653,0.5180,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.726,0.815,11,-4.969,1,0.0373,0.072400,0.004210,0.3570,0.6930,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.675,0.931,1,-3.432,0,0.0742,0.079400,0.000023,0.1100,0.6130,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.718,0.930,7,-3.778,1,0.1020,0.028700,0.000009,0.2040,0.2770,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,dance pop,0.650,0.833,1,-4.672,1,0.0359,0.080300,0.000000,0.0833,0.7250,123.976,189052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32828,7bxnKAamR3snQ1VGLuVfC1,City Of Lights - Official Radio Edit,Lush & Simon,42,2azRoBBWEEEYhqV6sb7JrT,City Of Lights (Vocal Mix),2014-04-28,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,0.428,0.922,2,-1.814,1,0.0936,0.076600,0.000000,0.0668,0.2100,128.170,204375
32829,5Aevni09Em4575077nkWHz,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20,6kD6KLxj7s8eCE3ABvAyf5,Closer Remixed,2013-03-08,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,0.522,0.786,0,-4.462,1,0.0420,0.001710,0.004270,0.3750,0.4000,128.041,353120
32830,7ImMqPP3Q1yfUHvsdn7wEo,Sweet Surrender - Radio Edit,Starkillers,14,0ltWNSY9JgxoIZO4VzuCa6,Sweet Surrender (Radio Edit),2014-04-21,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,0.529,0.821,6,-4.899,0,0.0481,0.108000,0.000001,0.1500,0.4360,127.989,210112
32831,2m69mhnfQ1Oq6lGtXuYhgX,Only For You - Maor Levi Remix,Mat Zo,15,1fGrOkHnHJcStl14zNx8Jy,Only For You (Remixes),2014-01-01,♥ EDM LOVE 2020,6jI1gFr6ANFtT8MmTvA2Ux,edm,progressive electro house,0.626,0.888,2,-3.361,1,0.1090,0.007920,0.127000,0.3430,0.3080,128.008,367432


## 1NF

The first normal form (1NF) requires the dataset has no repeating groups of columns and each columns must have a single value.

By checking each variables, I found that though the variable `track_name` and variable `track_album_name` are highly overlapped, but not totally repeated, so the original dataset doesn't violate the 1NF.  

## 2NF

The second normal form (2NF) requires all columns in each row depend fully on candidate keys(include simple and composite key), which means no column depends on the any subset of candiate keys.

We can check if the dataset conform 2NF by:

- find all possible candidate keys;
- check if other columns depend fully on all possible candidate keys.

If the dataset not satisfying 2NF, we just split the dataset into different table to make each table conform 2NF. To do so, we just need to:

- put those columns partially depend on candidate keys into new tables with the column(s) in candidate key they depend on;
- create linker tables if needed


Firstly, we need to find all the candidates keys. Technically, we need to find all possible combination of columns name, but if we do so, it will be extremely time consuming (combinatorial number is very terrible!). 

Fortunately, there are some clue in the columns names. We can find that only the first 11 columns determine the uniqueness of the song, and the rest columns are just attributes of the song.

Notice that there is no simple candiate keys, but there is a composite key with two columns, and it is the subset of all other possible keys, **which means we have only one candidate key: `track_id` and `playlist_subgenre`.** 

In [5]:
song_unique = ss.columns[0:11]
song_unique

Index(['track_id', 'track_name', 'track_artist', 'track_popularity',
       'track_album_id', 'track_album_name', 'track_album_release_date',
       'playlist_name', 'playlist_id', 'playlist_genre', 'playlist_subgenre'],
      dtype='object')

In [6]:
# https://blog.csdn.net/lanchunhui/article/details/51824602
# combinations() find all comb in a given list 
all_comb = [list(combinations(song_unique, x)) for x in range(1,11)]
all_comb = sum(all_comb, []) # to flatten the list 

In [7]:
# the number of comb
len(all_comb) 

2046

In [8]:
# the dict that how many duplicated value in each comb of columns
comb_dup = {x:ss.loc[:,x].duplicated().sum() for x in all_comb} 

In [9]:
# no sample key
[k for k, v in comb_dup.items() if v == 0 and len(k) == 1]

[]

In [10]:
# a composite key with two columns
[k for k, v in comb_dup.items() if v == 0 and len(k) == 2]

[('track_id', 'playlist_subgenre')]

In [11]:
# all other possible keys has subset to key_2, which means we only need to focus on key_2
key_2 = [k for k, v in comb_dup.items() if v == 0 and len(k) == 2][0]
all_possible_key = [k for k, v in comb_dup.items() if v == 0]
[k for k in all_possible_key if not set(key_2).issubset(k)]

[]

For the second step, since there is only one candiate key and the key is a composite key of two columns, we just need to check if the other columns depend partially on those two columns. To be specific, we just need to check for any rows with same values in columns `track_id` and `playlist_subgenre`, if values in other columns unqiue.

For example, the following means b not depends on a.

| a | b |
|---|---|
| $a_1$ | $b_1$ |
| $a_1$ | $b_2$ |

In [74]:
# https://www.coder.work/article/6843836
# https://stackoverflow.com/questions/41415017/count-unique-values-using-pandas-groupby/45091077
# the reference def a get_relation function, my function has similar logic
def get_relation(df, col1, col2):        
    first_max = df[[col1, col2]].groupby(col1).agg('nunique').max()[0]
    second_max = df[[col1, col2]].groupby(col2).agg('nunique').max()[0]
    if first_max==1:
        if second_max==1:
            return 'one-to-one'
        else:
            return 'one-to-many'
    else:
        if second_max==1:
            return 'many-to-one'
        else:
            return 'many-to-many'
        
def check_dependence(df, *cols):
    """
    This function can check the dependence of one column on other columns in a given dataframe.
    
    Parameters
    ----------
    df : pandas.DataFrame
        df is a pandas.DataFrame which the columns come from.
    cols : str
        The last col argument is the potential dependent, and the previous cols arguments are cols the last col may
        depend on.
        
    Returns
    -------
    bool
        True if the last column depends on the previous columns, False otherwise.
        
    Examples
    --------
    For example, there is a dataframe df which contains columns A, B and C, we want to check if the C depends on A 
    and B, we can use this function. Assume C depends on A and B, we should get a True.
    
    >>> print(check_dependence(df, A, B, C))
    True
    """
    cols_ = list(cols[:-1])
    dep = df.loc[:,cols].groupby(cols_).agg('nunique').max()[0]
    if dep == 1:
        return True
    else:
        return False

In [13]:
# check relationship of `track_id` and `playlist_subgenre`
get_relation(ss, *key_2)

'many-to-many'

In [14]:
nonekey_cols = list(ss.columns)
nonekey_cols.remove(key_2[0])
nonekey_cols.remove(key_2[1])

In [15]:
dep1 = {x: check_dependence(ss, key_2[0], x) for x in nonekey_cols}
dep1

{'track_name': True,
 'track_artist': True,
 'track_popularity': True,
 'track_album_id': True,
 'track_album_name': True,
 'track_album_release_date': True,
 'playlist_name': False,
 'playlist_id': False,
 'playlist_genre': False,
 'danceability': True,
 'energy': True,
 'key': True,
 'loudness': True,
 'mode': True,
 'speechiness': True,
 'acousticness': True,
 'instrumentalness': True,
 'liveness': True,
 'valence': True,
 'tempo': True,
 'duration_ms': True}

In [16]:
dep2 = {x: check_dependence(ss, key_2[1], x) for x in nonekey_cols}
dep2

{'track_name': False,
 'track_artist': False,
 'track_popularity': False,
 'track_album_id': False,
 'track_album_name': False,
 'track_album_release_date': False,
 'playlist_name': False,
 'playlist_id': False,
 'playlist_genre': True,
 'danceability': False,
 'energy': False,
 'key': False,
 'loudness': False,
 'mode': False,
 'speechiness': False,
 'acousticness': False,
 'instrumentalness': False,
 'liveness': False,
 'valence': False,
 'tempo': False,
 'duration_ms': False}

It is reasonable to see that all the all the `track_*` columns depend on `track_id`, and all the music attributions also depend on `track_id`. 

On the other hand, only `playlist_genre` depends on `playlist_subgenre`, which also makes perfect sense.

The above results show that the dataset does not satisfy 2NF.

To fix it, we just need to divide the dataset into three sub table: 

1. `ss_track`: `track_id` and columns that partially depend on it;
2. `ss_playlist_subgenre`: `playlist_subgenre` and columns that partially depend on it;
3. `ss_main`: `track_id`, `playlist_subgenre` and other columns that depend fully on these two columns;

Since we can use `ss_main` to pin down many to many relationship between `ss_track`, `ss_playlist_genre`, no linker table is needed.

In [17]:
# create cols name for each table
cols_1 = [x for x in nonekey_cols if dep1[x]]
cols_1.insert(0,key_2[0])

cols_2 = [x for x in nonekey_cols if dep2[x]]
cols_2.insert(0,key_2[1])

cols_3 = [x for x in nonekey_cols if not(dep1[x] or dep2[x])]
cols_3.insert(0,key_2[0])
cols_3.insert(1,key_2[1])

In [18]:
# create sub tables
ss_track = ss.loc[:, cols_1].drop_duplicates()
ss_playlist_subgenre = ss.loc[:, cols_2].drop_duplicates()
ss_main = ss.loc[:, cols_3].drop_duplicates()

## 3NF

The third normal form (3NF) requires no transitive dependence in each table.

To check this, we need to check the dependence relationship between columns in each tables. Since `ss_playlist_subgenre` only have two columns, so no transitive dependence exists. 

### `ss_track`

In [19]:
# create combinations of all possible pair which the last col is potential dependent
track_comb = sum([list(combinations(cols_1[:7], x)) for x in range(1,8)],[])
track_comb_ = [tuple(sum([list(x),[y]],[])) for x in track_comb for y in cols_1 if not y in x]
len(track_comb_)

1965

In [20]:
# check if the last col depends on the previous cols
dep_track_ = {x: check_dependence(ss_track, *x) for x in track_comb_}

In [21]:
# show which col depends on cols other than prime key
dep_track_true_ = {k[-1] for k, v in dep_track_.items() if v == True and 'track_id' not in k}
dep_track_true_

{'track_album_name', 'track_album_release_date', 'track_artist'}

In [22]:
# try to find what cols they depend on
{k:v for k, v in dep_track_.items() if v == True and 'track_id' not in k and len(k) <=3}

{('track_album_id', 'track_album_name'): True,
 ('track_album_id', 'track_album_release_date'): True,
 ('track_name', 'track_album_id', 'track_artist'): True,
 ('track_name', 'track_album_id', 'track_album_name'): True,
 ('track_name', 'track_album_id', 'track_album_release_date'): True,
 ('track_artist', 'track_album_id', 'track_album_name'): True,
 ('track_artist', 'track_album_id', 'track_album_release_date'): True,
 ('track_popularity', 'track_album_id', 'track_album_name'): True,
 ('track_popularity', 'track_album_id', 'track_album_release_date'): True,
 ('track_album_id', 'track_album_name', 'track_album_release_date'): True,
 ('track_album_id', 'track_album_release_date', 'track_album_name'): True}

In [23]:
# confirm that all other dependence are redundant
d1 = ('track_album_id', 'track_album_name')
d2 = ('track_album_id', 'track_album_release_date')
d3 = ('track_name', 'track_album_id', 'track_artist')
{k:v for k, v in dep_track_.items() 
 if v == True 
 and 'track_id' not in k 
 and not (set(d1).issubset(k) or set(d2).issubset(k) or set(d3).issubset(k))}

{}

### `ss_main`

In [24]:
# create combinations of all possible pair which the last col is potential dependent
pl2_comb = sum([list(combinations(cols_3, x)) for x in range(1,4)],[])
pl2_comb_ = [tuple(sum([list(x),[y]],[])) for x in pl2_comb for y in cols_3 if not y in x]
len(pl2_comb_)

28

In [25]:
# check if the last col depends on the previous cols
dep_pl2_ = {x: check_dependence(ss_main, *x) for x in pl2_comb_}

In [26]:
# find the cols that depend on cols other than prime key
dep_playlist2_true = {k: v for k, v in dep_pl2_.items() if v == True and not set(key_2).issubset(k)}
dep_playlist2_true

{('playlist_id', 'playlist_name'): True,
 ('track_id', 'playlist_name', 'playlist_id'): True,
 ('track_id', 'playlist_id', 'playlist_name'): True,
 ('playlist_subgenre', 'playlist_id', 'playlist_name'): True}

Base on above results, the three sub tables don't satisfy 3NF.

To fix it, we just need to:
- put the cols with transitive dependence into new tables with the cols they depend on other than prime key.


In [27]:
# split ss_track
tran_cols = ['track_album_name', 'track_album_release_date', 'track_artist']
ss_track_ = ss_track[[x for x in cols_1 if x not in tran_cols]].drop_duplicates()
ss_track_track_album_id = ss_track[['track_album_id', 'track_album_name', 'track_album_release_date']].drop_duplicates()
ss_track_track_album_id_track_name = ss_track[['track_name', 'track_album_id', 'track_artist']].drop_duplicates()

In [28]:
# split ss_main
ss_main_ = ss_main[['track_id', 'playlist_subgenre', 'playlist_id']].drop_duplicates()
ss_playlist_id = ss_main[['playlist_id', 'playlist_name']].drop_duplicates()

## Save as SQLite3 schema

Now we have all the sub tables:

- `ss_track_`, `ss_track_track_album_id`, `ss_track_track_album_id_track_name`;
- `ss_playlist_subgenre`;
- `ss_main`, `ss_playlist_id`;

Then we need to save them into a SQLite3 schema. 

In [38]:
# save tables into SQLite3 schema
engine = create_engine('sqlite:///ss.sqlite', echo = False)
ss_track_.to_sql('track', con = engine, if_exists='replace', index = False)
ss_track_track_album_id.to_sql('track_album_id', con = engine, if_exists='replace', index = False)
ss_track_track_album_id_track_name.to_sql('track_album_id_track_name', con = engine, if_exists='replace', index = False)
ss_playlist_subgenre.to_sql('playlist_subgenre', con = engine, if_exists='replace', index = False)
ss_main_.to_sql('main', con = engine, if_exists='replace', index = False)
ss_playlist_id.to_sql('playlist_id', con = engine, if_exists='replace', index = False)

In [39]:
if not os.path.exists('ss.png'):
    render_er('sqlite:///ss.sqlite', 'ss.png')
else:
    ! rm ss.png
    render_er('sqlite:///ss.sqlite', 'ss.png')

## \*Create a SQLite3 schemas that tables contain keys

In sqlite, the usage of `ALTER TABLE` is very limited, and can not be use to add or drop keys. The only way to add or drop key is to recreate the tables with keys which seems very clumsy. If we use `to_sql` directly, we will end up with tables that without keys

To demonstrate the structure of tables, I will create tables with key and save them into a SQLite3 schame (since the purpose is to show the structure, there is no need to insert data, while inserting value is very simple using `to_sql(..., if_exists='append')`).


In [31]:
engine = create_engine('sqlite:///ss_structure_.sqlite', echo = False)

In [33]:
%sql sqlite:///ss_structure_.sqlite

In [34]:
%%sql

DROP TABLE IF EXISTS track;
DROP TABLE IF EXISTS track_album_id;
DROP TABLE IF EXISTS track_album_id_track_name;
DROP TABLE IF EXISTS playlist_subgenre;
DROP TABLE IF EXISTS main;
DROP TABLE IF EXISTS playlist_id;

CREATE TABLE main (
    track_id TEXT,
    playlist_subgenre TEXT,
    playlist_id TEXT,
    PRIMARY KEY (track_id, playlist_subgenre)
);

CREATE TABLE playlist_id (
    playlist_id TEXT PRIMARY KEY,
    playlist_name TEXT,
    FOREIGN KEY (playlist_id) 
      REFERENCES main (playlist_id)
);

CREATE TABLE track (
    track_id TEXT PRIMARY KEY,
    track_name TEXT,
    track_popularity INTEGER,
    track_album_id TEXT,
    danceability REAL,
    energy REAL,
    key INTEGER,
    loudness REAL,
    mode INTEGER,
    speechiness REAL,
    acousticness REAL,
    instrumentalness REAL,
    liveness REAL,
    valence REAL,
    tempo REAL,
    duration_ms INTEGER,
    FOREIGN KEY (track_id) 
      REFERENCES main (track_id)
);

CREATE TABLE track_album_id (
    track_album_id TEXT PRIMARY KEY,
    track_album_name TEXT,
    trac_alblum_release_date TEXT,
    FOREIGN KEY (track_album_id) 
      REFERENCES track (track_id)
);

CREATE TABLE track_album_id_track_name (
    track_album_id TEXT,
    track_name TEXT,
    track_artist TEXT,
    PRIMARY KEY (track_album_id, track_name),
    FOREIGN KEY (track_album_id, track_name) 
      REFERENCES track (track_album_id, track_name)
);

CREATE TABLE playlist_subgenre (
    playlist_subgenre TEXT PRIMARY KEY,
    playlist_genre TEXT,
    FOREIGN KEY (playlist_subgenre) 
      REFERENCES main (playlist_subgenre)
);

 * sqlite:///ss_structure_.sqlite
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [35]:
if not os.path.exists('ss_structure_.png'):
    render_er('sqlite:///ss_structure_.sqlite', 'ss_structure_.png')
else:
    ! rm ss_structure_.png
    render_er('sqlite:///ss_structure_.sqlite', 'ss_structure_.png')

![ss_structure_](ss_structure_.png)

## SQL query

Now we can use the SQL query to find the names of all playlists that contain instrumentals.

By the definition of column `instrumentalness`:

>  Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.

We can see that higher `instrumentalness` represents higher probability of being a instrumental track. I will use 0 and 0.5 as threshold (above 0 means contain instrumentals, above 0.5 are intended to represent instrumental tracks).

In [32]:
%load_ext sql

In [41]:
%sql sqlite:///ss.sqlite

In [71]:
%%sql

SELECT DISTINCT playlist_name
FROM playlist_id
INNER JOIN main
    ON playlist_id.playlist_id = main.playlist_id
INNER JOIN track
    ON main.track_id = track.track_id
WHERE instrumentalness > 0

 * sqlite:///ss.sqlite
   sqlite:///ss_structure_.sqlite
Done.


playlist_name
Pop Remix
EDM House & Dance
Big Room House / Bigroom
Pop EDM Remixes
"Festival Music 2019 - Warm Up Music (EDM, Big Room & Progressive House)"
Trance Party 2019 by FUTURE TRANCE
EDM 2020 House & Dance
"post-teen alternative, indie, pop (large variety)"
New House ‍
Dancefloor Beats


In [73]:
%%sql

SELECT DISTINCT playlist_name
FROM playlist_id
INNER JOIN main
    ON playlist_id.playlist_id = main.playlist_id
INNER JOIN track
    ON main.track_id = track.track_id
WHERE instrumentalness > 0.5

 * sqlite:///ss.sqlite
   sqlite:///ss_structure_.sqlite
Done.


playlist_name
Pop Remix
Dance Room
Pop Warmup 130 BPM
Dance Pop
Dance Pop Tunes
Pop / Dance
Most Popular 2020 TOP 50
"post-teen alternative, indie, pop (large variety)"
Todo Éxitos
Charts 2020 🔥Top 2020🔥Hits 2020🔥Summer 2020🔥Pop 2020🔥Popular Music🔥Clean Pop 2020🔥Sing Alongs


## Reference
1. http://www.gitta.info/LogicModelin/en/html/DataConsiten_Norm3NF.html
2. https://www.sqlitetutorial.net/sqlite-create-table/